In [2]:
import pandas as pd
import numpy as np
import json
import datetime
import teradata
import logging
import os
import xlwt
import pickle
import re
import pyodbc
import warnings
import snowflake.connector
from tqdm.notebook import tqdm, trange 
warnings.filterwarnings("ignore")

now = datetime.datetime.now()
yesterday = (now - datetime.timedelta(days = 1)).strftime('%Y%m%d')
today = (now).strftime('%Y%m%d')
print('Today:\t\t',today)
print('Yesterday:\t',yesterday)

Today:		 20241119
Yesterday:	 20241118


In [3]:
# Snowflake connection parameters
params_dev = {
    'user': 'malcolm.udoh@betmgm.com',
    'account': 'ucssray-edp_us_dev',
    'authenticator': 'externalbrowser',  # Use externalbrowser as the authenticator for SSO
    'warehouse': 'WH_BETMGM_DEV',  # Specify your Snowflake warehouse
    'database': 'EDLPRODVIEWS',  # Specify your Snowflake database
    'role': 'RL_DEV_USBI_RED_R'
    
}
params = {
    'user': 'malcolm.udoh@betmgm.com',
    'account': 'ucssray-edp_us_prod',
    'authenticator': 'externalbrowser',  # Use externalbrowser as the authenticator for SSO
    'warehouse': 'WH_US_PRODBI',  # Specify your Snowflake warehouse
    'database': 'EDLPRODVIEWS',  # Specify your Snowflake database
    'role': 'RL_PROD_USBI_RED_R'
    
}
conn_sf = snowflake.connector.connect(**params)
conn_sf_dev = snowflake.connector.connect(**params_dev)

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/60c43c0a-64ac-4050-bf3e-31e1cdfffdeb/saml2?SAMLRequest=nZJdb9owFIb%2FSuRdJ7GTFIoFVAyEhtaPqMCmcWecE%2FBw7NR2SNmvnwlF6i7ai91Z9nOOH%2Fs9w7vXSgZHMFZoNUIkwigAxXUh1G6E1qt5eIsC65gqmNQKRugEFt2Nh5ZVsqaTxu3VM7w0YF3gGylLu4MRaoyimllhqWIVWOo4XU4e7mkSYVob7TTXEr0r%2BbyCWQvGecNrSWGF19s7V9M4bts2atNIm12cYIxjPIg9dUa%2BXPlX%2F6YPeBLj7Mx7wuP5m9tXoS5f8JnW9gJZ%2Bm21ysP8ablCweSqOtXKNhWYJZij4LB%2Bvr8IWG%2BgIclu%2B7dRY0MOyhkmSbTjdWSVbkvJDsB1VTfO9478Ki6hiKXeCf%2F8xWyE6oMoTr3Bway%2B%2F9o%2BzH%2BqdALL9PHp92b%2Fss738oBLtsnbrTpOyZ%2FMchT8uOabnPNdWNvAQp1TdX4LJ1lISEgGK5LQrE%2FTQTTo32xQMPOpCsVcV3lV7zyiSnCjrS6dVlIo6Cx7mGcpxyzsZYyHGb7B4bZMIUwJEF6UZVnANj5nnaDL%2FNBOxIz%2F71eG8fsebwP56DNazHItBT8Fc20q5j6OkESk2xFFWHYohYoJOSkKA9b6KKXU7dQAc37unWkAxe

### Queries

## Monthly

In [ ]:
monthly_list = '''
Select distinct dc.month_short_desc,
    dc.MONTH_OF_YEAR
from EDLPRODVIEWS.EDLPRODVIEWSUSABI.DIM_CALENDAR dc
where 1 = 1
    and year_of_calendar = 2023
    Order by 2
    '''
df_cal_month_abbre = pd.read_sql(monthly_list, conn_sf)
print(df_cal_month_abbre.columns)
df_cal_month_abbre.head()

In [ ]:
for a in tqdm(df_cal_month_abbre.values):
    month = a[0]df_cal_month_abbre.values.(str.lower())
    df_cal_month_abbre.columns = map(str.upper, df_cal_month_abbre.columns)

In [ ]:
monthly_query = '''
Select distinct dc.CALENDAR_DATE,
from EDLPRODVIEWS.EDLPRODVIEWSUSABI.DIM_CALENDAR dc
where 1 = 1
    and year_of_calendar = 2023
    and to_varchar(lower(dc.month_short_desc)) = '**month**'
    order by 1
    '''
df_cal_month = pd.read_sql(monthly_query.replace('**month**',month), conn_sf)
print(df_cal_month.columns)
df_cal_month.head()

## Weekly

In [ ]:
weekly_query = '''
select distinct dc.WEEK_OF_YEAR,
                case
                    when start_of_this_week_date < '2023-01-01' then '2023-01-01'
                    else start_of_this_week_date
                end start_of_this_week_date,
                case
                    when end_of_this_week_date > '2023-12-31' then '2023-12-31'
                    else end_of_this_week_date
              end end_of_this_week_date   
from EDLPRODVIEWS.EDLPRODVIEWSUSABI.DIM_CALENDAR dc
where 1 = 1
    and year_of_calendar = 2023
order by week_of_year
'''
df_cal_week = pd.read_sql(weekly_query, conn_sf)
print(df_cal_week.columns)
df_cal_week.head()

## Daily

In [ ]:
daily_query = '''
Select distinct dc.CALENDAR_DATE,
 from EDLPRODVIEWS.EDLPRODVIEWSUSABI.DIM_CALENDAR dc
 where 1 = 1
     and year_of_calendar = 2023
    and to_varchar(lower(dc.month_short_desc)) = 'feb'
     order by 1
     --DESC
    '''
df_cal_day = pd.read_sql(daily_query, conn_sf)
print(df_cal_day.columns)
df_cal_day.head()

In [ ]:
daily_query = '''
Select distinct dc.CALENDAR_DATE,
from EDLPRODVIEWS.EDLPRODVIEWSUSABI.DIM_CALENDAR dc
where 1 = 1
    and year_of_calendar = 2023
    order by 1
    '''
df_cal_day = pd.read_sql(daily_query, conn_sf)
print(df_cal_day.columns)
df_cal_day.head()

## Demographics Data

In [ ]:
query = '''
with test_account_player_id as (
select distinct dp.player_id
from EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_player dp
inner join EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_player_account dpa
    on dp.player_id = dpa.player_id
left join EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_cashier_category dcc
    on dpa.wh_cashier_category_cd = dcc.wh_cashier_category_cd
left join EDLPRODVIEWS.EDLPRODVIEWSUSABI.player_extension pe
    on dp.player_id = pe.player_id
left join EDLPRODVIEWS.EDLPRODVIEWSUSABI.DIM_PLAYER_PII dpi
    on dp.player_id = dpi.player_id
where 1 = 1
    and dcc.Cashier_Category_Desc not in ('Closed','PermanentlyClosed','PermenantlyClosed')
    and (dcc.cashier_category_desc in ('TestAccount')
         or
         trim(lower(pe.test_account_yn)) = 'y'
         or
         trim(lower(cast(dpi.email_id_txt as varchar))) like ('%bwinparty%')
         or trim(lower(cast(dpi.email_id_txt as varchar))) like ('%ivycomptech%')
         or trim(lower(cast(dpi.email_id_txt as varchar))) like ('%gvcgroup%')
         or trim(lower(cast(dpi.email_id_txt as varchar))) like ('%yopmail%')
         or trim(lower(cast(dpi.email_id_txt as varchar))) like ('%mgm%')
         or trim(lower(cast(dpi.email_id_txt as varchar))) like ('%ladbrokescoral1%')
         or trim(lower(cast(dpi.email_id_txt as varchar))) like ('%internalgvc%')
         or trim(lower(cast(dp.Login_Name_Txt as varchar))) like ('%beta')
         or trim(lower(cast(dp.Login_Name_Txt as varchar))) like ('%test')
         )
)

select --dp.player_id, dp.login_name_txt,
       dupi.dupi,
       to_char(dpi.date_of_birth, 'MM') as birth_month,
       to_char(dpi.date_of_birth, 'YYYY') as birth_year,
       dpi.gender_cd as gender,
       dpi.state_txt as state,
       dpi.zip_txt as zipcode,
       dp.registration_timestamp as Signup_dt,
       'BetMGM' as Venue,
       dp.primary_brand_cd as Website
from EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_player dp
inner join EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_skin ds
    on dp.registration_wh_skin_id = ds.wh_skin_id
    and ds.state = ('NJ')
    and dp.front_end_cd in ('mj','bj','pj','wo')
left join EDLPRODVIEWS.EDLPRODVIEWSUSABI.DIM_PLAYER_PII dpi
    on dp.player_id = dpi.player_id
left join EDLPRODVIEWS.EDLPRODVIEWSUSABI.PLAYER_DUPI_UAT dupi
    on dp.player_id = dupi.player_id
left join EDLPRODVIEWS.EDLPRODVIEWSUSABI.CUSTOMER_PLAYER_ASSOCIATION_LOG cpal
    on dp.player_id = cpal.player_id
where 1 = 1
    and ((cpal.is_master_account = 1 and dp.front_end_cd = 'mj') or
         dp.front_end_cd <> 'mj')
    and dp.registration_date <= '2023-12-31'
    --and dp.registration_date between '2023-08-01' and '2023-12-31'
    and dp.player_id not in (select *
                             from test_account_player_id)
order by 7 desc
'''    
df_demographics = pd.read_sql(query, conn_sf_dev)
df_demographics.columns = map(str.upper, df_demographics.columns)

number_of_row = 1048576
loop_count = int(len(df_demographics) / number_of_row)

for cycle in tqdm(range(loop_count)):
#     df_demographics[cycle*number_of_row:(cycle+1)*number_of_row].to_csv(r'C:\Users\malcolm.udoh\Output_Files\Rutgers_data_pull_2023\BetMGM_NJ_2023_Demographics_1_half.csv'.format(cycle))
    df_demographics = df_demographics[['DUPI','BIRTH_MONTH', 'BIRTH_YEAR', 'GENDER', 'STATE', 'ZIPCODE', 'SIGNUP_DT', 'VENUE', 'WEBSITE']]
    df_demographics.to_csv(r'C:\Users\malcolm.udoh\Output_Files\Rutgers_data_pull_2023\Rutgers_BetMGM_NJ_2023_Demographics_' + str(cycle)  + '.csv', index = False)
#df_demographics.to_csv(r'C:\Users\malcolm.udoh\Output_Files\Rutgers_data_pull_2023\BetMGM_NJ_2023_Demographics_2_half.csv', index = False)
print(df_demographics.info(),'\n')
df_demographics.head()

### Login/Logout Info

In [ ]:
query = '''
with casino_bets as (
select fc.player_id, 
      fc.game_start_timestamp,
      fc.game_end_timestamp,
      1 as Played
from edlprodviews.edlprodviewsusabi.f_player_casino_txn fc
inner join edlprodviews.edlprodviewsusabi.dim_skin ds
   on fc.wh_skin_id = ds.wh_skin_id
   and ds.state = ('NJ')
   and ds.front_end_cd in ('mj','bj','pj','wo')
where 1 = 1
--and fc.game_start_timestamp >= '2023-01-01 00:00:00' 
    --and fc.game_end_timestamp <= '2023-01-07 23:59:59'
     and fc.game_start_timestamp <= timestamp '**end_dt** 23:59:59'
     and fc.game_end_timestamp >= timestamp '**beg_dt** 00:00:00'
    --and fc.txn_date between DATE('2023-01-01 00:00:00') and DATE('2023-01-07 23:59:59')
),
sports_bets as (
select fs.player_id, 
      fs.evaluation_timestamp,
      null,
      1 as Played
from edlprodviews.edlprodviewsusabi.f_player_sports_txn fs
inner join edlprodviews.edlprodviewsusabi.dim_skin ds
   on fs.wh_skin_id = ds.wh_skin_id
   and ds.state = ('NJ')
   and ds.front_end_cd in ('mj','bj','pj','wo')
where 1 = 1
and fs.sports_txn_type_id = 1
    --and fs.evaluation_timestamp between '2023-01-01 00:00:00' and '2023-01-07 23:59:59'
   -- and fs.summary_date between '2023-01-01' and '2023-01-07'
and fs.evaluation_timestamp between
 	timestamp '**beg_dt** 00:00:00' and timestamp '**end_dt** 23:59:59'
),
bets as
(Select * FROM casino_bets 
    UNION ALL SELECT * FROM sports_bets)
 
 
select distinct dupi.dupi,
       a.session_start_timestamp as login_dt,
       case
           when a.session_end_timestamp > current_timestamp then DateAdd(Hour,1,a.session_start_timestamp)
           else a.session_end_timestamp
       end as logout_dt,
       1 as PLAYED
from edlprodviews.edlprodviewsusabi.player_session a
inner join edlprodviews.edlprodviewsusabi.dim_player dp
    on a.player_id = dp.player_id
inner join edlprodviews.edlprodviewsusabi.dim_skin ds
    on dp.registration_wh_skin_id = ds.wh_skin_id
    and ds.state = ('NJ')
    and dp.front_end_cd in ('mj','bj','pj', 'wo')
left join edlprodviews.edlprodviewsusabi.PLAYER_DUPI_UAT dupi
    on dp.player_id = dupi.player_id
inner join bets b
    on a.player_id = b.player_id
    and b.game_start_timestamp between
                 a.session_start_timestamp and a.session_end_timestamp
where 1 = 1
--and  a.SESSION_START_TIMESTAMP between '2023-01-01 00:00:00' and '2023-01-07 23:59:59'
and a.session_start_timestamp between 
       timestamp '**beg_dt** 00:00:00' and timestamp '**end_dt** 23:59:59'
-- and dp.registration_date between '2023-01-01' and '2023-01-08'
   and dp.registration_date <= date('2023-12-31')
    and dupi.dupi is not null
 
union
select distinct dupi.dupi,
       a.session_start_timestamp as login_dt,
       case
           when a.session_end_timestamp > current_timestamp then DateAdd(Hour,1,a.session_start_timestamp)
           else a.session_end_timestamp
       end as logout_dt,
       0 as PLAYED
       --case
       --		when cb.played = 1 or sb.played = 1 then 1
       --		else 0
       --end as Played
from edlprodviews.edlprodviewsusabi.player_session a
inner join edlprodviews.edlprodviewsusabi.dim_player dp
    on a.player_id = dp.player_id
inner join edlprodviews.edlprodviewsusabi.dim_skin ds
    on dp.registration_wh_skin_id = ds.wh_skin_id
    and ds.state = ('NJ')
    and dp.front_end_cd in ('mj','bj','pj','wo')
left join edlprodviews.edlprodviewsusabi.PLAYER_DUPI_UAT dupi
    on dp.player_id = dupi.player_id
where 1 = 1
and a.session_start_timestamp between 
       timestamp '**beg_dt** 00:00:00' and timestamp '**end_dt** 23:59:59'
--and  a.SESSION_START_TIMESTAMP between '2023-01-01 00:00:00' and '2023-01-07 23:59:59'
    and dp.registration_date <= date('2023-12-31')
    and dupi.dupi is not null
    and (not exists (select 1
                 from casino_bets b
                 where 1 = 1
                     and a.player_id = b.player_id
                     and b.game_start_timestamp between
                     a.session_start_timestamp and a.session_end_timestamp) and 
         not exists (select 1
                 from sports_bets c
                 where 1 = 1
                     and a.player_id = c.player_id
                     and c.evaluation_timestamp between
                     a.session_start_timestamp and a.session_end_timestamp))
                     '''
for x in tqdm(df_cal_week.values):
    beg_dt = x[1].strftime('%Y-%m-%d')
    end_dt = x[2].strftime('%Y-%m-%d')
    df_login = pd.read_sql(query.replace('**beg_dt**',beg_dt).replace('**end_dt**',end_dt), conn_sf_dev)
    df_login.columns = map(str.upper, df_login.columns)
    #print(df_login.shape[0],'\n')
    df_login.to_csv(r'C:\Users\malcolm.udoh\Output_Files\Rutgers_data_pull_2023\Rutgers_Login_2023_weekly_breakdown\2023_Rutgers_BetMGM_NJ_Login_' + beg_dt.replace('-','') + '_' + end_dt.replace('-','')  + '.csv', index = False)

### RG Features

In [ ]:
query = '''    
with test_account_player_id as (
select distinct dp.player_id
from EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_player dp
inner join EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_player_account dpa
    on dp.player_id = dpa.player_id
left join EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_cashier_category dcc
    on dpa.wh_cashier_category_cd = dcc.wh_cashier_category_cd
left join EDLPRODVIEWS.EDLPRODVIEWSUSABI.player_extension pe
    on dp.player_id = pe.player_id
left join EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_player_pii dpi
    on dp.player_id = dpi.player_id
where 1 = 1
    and dcc.Cashier_Category_Desc not in ('Closed','PermanentlyClosed','PermenantlyClosed')
    and (dcc.cashier_category_desc in ('TestAccount')
         or trim(lower(to_varchar(pe.test_account_yn))) = 'y'
         or trim(lower(to_varchar(dpi.email_domain_txt))) like ('%bwinparty%')
         or trim(lower(to_varchar(dpi.email_domain_txt))) like ('%ivycomptech%')
         or trim(lower(to_varchar(dpi.email_domain_txt))) like ('%gvcgroup%')
         or trim(lower(to_varchar(dpi.email_domain_txt))) like ('%yopmail%')
         or trim(lower(to_varchar(dpi.email_domain_txt))) like ('%mgm%')
         or trim(lower(to_varchar(dpi.email_domain_txt))) like ('%ladbrokescoral1%')
         or trim(lower(to_varchar(dpi.email_domain_txt))) like ('%internalgvc%')
         or trim(lower(to_varchar(dp.Login_Name_Txt))) like ('%beta')
         or trim(lower(to_varchar(dp.Login_Name_Txt))) like ('%test')
         )
)

select x.dupi,

       x.created_dt,
       x.limit_type,
       x.limit_set_period,
       x.amount_units,
       case
       	when x.rownum = 1 then 'Initial'
       	when x.rownum <> 1 and x.lag_amt > x.amount_units then 'Decrease'
       	when x.rownum <> 1 and x.lag_amt < x.amount_units then 'Increase'
       end as limit_purpose     
from (
select dupi.dupi,
dp.login_name_txt,
       a.TXN_TIMESTAMP as created_dt,
       'Deposit' as limit_type,
       case 
       	when a.period_days = 1 then 'Daily'
       	when a.period_days = 7 then 'Weekly'
       	when a.period_days = 30 then 'Monthly'
       end as limit_set_period,
       a.max_deposit_amt_tc as amount_units,
       rank() over (partition by a.player_id, a.period_days
                    order by a.TXN_TIMESTAMP) as rownum,
       lag(a.max_deposit_amt_tc) over (partition by a.player_id, a.period_days
                                        order by a.TXN_TIMESTAMP) as lag_amt
from EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_player_rg_deposit_limit a
inner join EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_player dp
	on a.player_id = dp.player_id
inner join EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_skin ds
    on dp.registration_wh_skin_id = ds.wh_skin_id
    and ds.state = ('NJ')
    and ds.front_end_cd in ('mj','bj','pj', 'wo')
left join EDLPRODVIEWS.EDLPRODVIEWSUSABI.PLAYER_DUPI_UAT dupi
    on dp.player_id = dupi.player_id
where 1 = 1
    and a.period_days = 1
    and a.status_cd = 'Verified'
    and dp.registration_date <= '2023-12-31'
    and a.TXN_TIMESTAMP <= '2023-12-31 23:59:59'
    and a.player_id not in (Select Player_Id from EDLPRODVIEWS.EDLPRODVIEWSUSABI.Dim_Test_Account ta where ta.Player_Id = a.Player_Id)             
) x
order by 1, 4, 2
'''

df_limit = pd.read_sql(query, conn_sf_dev)
df_limit.columns = map(str.upper, df_limit.columns)
print(df_limit.shape[0],'\n')
df_limit.to_csv(r'C:\Users\malcolm.udoh\Output_Files\Rutgers_data_pull_2023\2023_Rutgers_BetMGM_NJ_RG_Features_Deposit_Limit.csv', index = False)
df_limit.head()

## Oracle

In [ ]:
import getpass

import oracledb

In [ ]:
host = 'nj0d1udora444.dev.nj03.bpty'
port = '1521'
sid = 'REALNJDEVSRVD'
sid = oracledb.makedsn(host, port, service_name=sid)

connection = oracledb.connect(
    user = 'ezecash',
    password = 'ezecash',
    dsn = '//nj0d1udora444.dev.nj03.bpty:1521/REALNJDEVSRVD')
cursor = connection.cursor()
print('pass:', sid)

### Time/Session limits

In [ ]:
query_time_limit = '''
SELECT x.F_ACCOUNT_NAME AS LOGIN_NAME_TXT,
       F_VERIFIED_DATE AS CREATED_DT,
       'Play time limit' AS Limit_Type,
       'daily' AS Limit_set_period,
       CASE
       	WHEN f_session_time_limit > 0 THEN f_session_time_limit/ 60
       	ELSE 0
       END AS amount_units,
       CASE 
       	WHEN rnk = 1 THEN 'Initial'
       	WHEN rnk > 1 AND x.F_SESSION_TIME_LIMIT > x.LAG_VAL THEN 'Increase'
       	WHEN rnk > 1 AND x.F_SESSION_TIME_LIMIT < x.LAG_VAL THEN 'Decrease'
       	ELSE 'No Change'
       END AS Limit_purpose
FROM (
	SELECT a.*,
	       row_number() OVER (PARTITION BY f_account_name ORDER BY f_verified_date asc) rnk,
	       lag(F_SESSION_TIME_LIMIT) OVER (PARTITION BY f_account_name ORDER BY f_verified_date asc) lag_val
	FROM ara.t_rg_session_limit_log a
	INNER JOIN ezecash.t_g_biographic b
	    ON a.f_account_name = b.f_account_name_eze
	    AND b.f_category_id <> 22
	WHERE 1 = 1
	    AND a.f_verified_date <= to_date('2023-12-31 23:59:59', 'YYYY-MM-DD hh24:mi:ss')
	    AND a.f_status = 'verified'
	    --AND a.f_session_time_limit > 0
	    AND a.f_account_name NOT LIKE '%test%'
	 ) x
WHERE 1 = 1
	--AND rownum < 100
	AND x.f_verified_date >= to_date('2023-01-01 00:00:00', 'YYYY-MM-DD hh24:mi:ss')
ORDER BY F_ACCOUNT_NAME, f_verified_date
'''

df_time_limit = pd.read_sql(query_time_limit, connection)
df_time_limit.columns = map(str.upper, df_time_limit.columns)
df_time_limit['LNT'] = "'" + df_time_limit['LOGIN_NAME_TXT'] + "'"
print(df_time_limit.shape[0],'\n')
pl_qry = str(df_time_limit['LNT'].to_list()).replace('[','').replace(']','').replace("'","")
df_time_limit.head()

In [ ]:
query = '''
select dp.login_name_txt, dupi.*
from EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_player dp
left join EDLPRODVIEWS.EDLPRODVIEWSUSABI.PLAYER_DUPI_UAT dupi
    on dp.player_id = dupi.player_id
where 1 = 1
    and dp.login_name_txt in (**pl_qrt**)
'''

df_ply_dupi = pd.read_sql(query.replace('**pl_qrt**',pl_qry).replace('"',"'"), conn_sf_dev)
df_ply_dupi.columns = map(str.upper, df_ply_dupi.columns)
print(df_ply_dupi.shape[0],'\n')

In [ ]:
df_time_limit_final = df_time_limit.merge(df_ply_dupi, on = 'LOGIN_NAME_TXT', how = 'left')
df_time_limit_final = df_time_limit_final[['DUPI','CREATED_DT','LIMIT_TYPE','LIMIT_SET_PERIOD','AMOUNT_UNITS','LIMIT_PURPOSE']]
df_time_limit_final.to_csv(r'C:\Users\malcolm.udoh\Output_Files\Rutgers_data_pull_2023\Rutgers_BetMGM_NJ_2023_RG_Features_Time_Limit.csv', index = False)

### Spend/ Loss Limits

In [ ]:
query_spend_limit = '''
SELECT x.F_USER_ID AS LOGIN_NAME_TXT,
       to_char(F_LAST_UPDATED, 'YYYY-MM-DD hh24:mi:ss') AS CREATED_DT,
       'Spend money (loss) limit' AS Limit_Type,
       CASE
       	WHEN x.f_days = 1 THEN 'daily'
       	WHEN x.f_days = 7 THEN 'weekly'
       	WHEN x.f_days = 30 THEN 'monthly'
       END AS Limit_set_period,
       (F_LIMIT_VALUE/ 100) AS amount_units,
       CASE 
       	WHEN rnk = 1 THEN 'Initial'
       	WHEN rnk > 1 AND x.F_LIMIT_VALUE > x.LAG_VAL THEN 'Increase'
       	WHEN rnk > 1 AND x.F_LIMIT_VALUE < x.LAG_VAL THEN 'Decrease'
       	ELSE 'No Change'
       END AS Limit_purpose
FROM (
	SELECT a.*,
	       row_number() OVER (PARTITION BY f_user_id, f_days ORDER BY F_LAST_UPDATED asc) rnk,
	       lag(F_LIMIT_VALUE) OVER (PARTITION BY f_user_id, f_days ORDER BY F_LAST_UPDATED asc) lag_val
	FROM Ara.T_Rg_Limits_Log a
	INNER JOIN ezecash.t_g_biographic b
	    ON a.F_USER_ID = b.f_account_name_eze
	    AND b.f_category_id <> 22
	WHERE 1 = 1
	    --AND a.F_LAST_UPDATED  <= to_date('2022-12-31 23:59:59', 'YYYY-MM-DD hh24:mi:ss')
        AND a.F_LAST_UPDATED between to_date('2023-01-01 00:00:00', 'YYYY-MM-DD hh24:mi:ss') and to_date('2023-12-31 23:59:59', 'YYYY-MM-DD hh24:mi:ss')
	    AND trim(lower(a.f_status_id)) = 'verified'
	    --AND a.f_session_time_limit > 0
	    AND F_USER_ID NOT LIKE '%test%'
        AND F_USER_ID NOT LIKE '%beta%'
	 ) x
WHERE 1 = 1
	--AND rownum < 100
	AND x.F_LAST_UPDATED >= to_date('2022-01-01 00:00:00', 'YYYY-MM-DD hh24:mi:ss')
ORDER BY F_USER_ID, F_LAST_UPDATED
'''

df_loss_limit = pd.read_sql(query_spend_limit, connection)
df_loss_limit.columns = map(str.upper, df_loss_limit.columns)
df_loss_limit['LNT'] = "'" + df_loss_limit['LOGIN_NAME_TXT'] + "'"
print(df_loss_limit.shape[0],'\n')
pl_qry = str(df_loss_limit['LNT'].to_list()).replace('[','').replace(']','').replace("'","")
df_loss_limit.head()

In [ ]:
query = '''
select dp.login_name_txt, dupi.*
from EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_player dp
left join EDLPRODVIEWS.EDLPRODVIEWSUSABI.PLAYER_DUPI_UAT dupi
    on dp.player_id = dupi.player_id
where 1 = 1
    and dp.login_name_txt in (**pl_qrt**)
order by 1
'''

df_ply_dupi = pd.read_sql(query.replace('**pl_qrt**',pl_qry).replace('"',"'"), conn_sf_dev)
df_ply_dupi.columns = map(str.upper, df_ply_dupi.columns)
print(df_ply_dupi.shape[0],'\n')

In [ ]:
df_loss_limit_final = df_loss_limit.merge(df_ply_dupi, on = 'LOGIN_NAME_TXT', how = 'left')
df_loss_limit_final = df_loss_limit_final[['DUPI','LOGIN_NAME_TXT', 'CREATED_DT','LIMIT_TYPE','LIMIT_SET_PERIOD','AMOUNT_UNITS','LIMIT_PURPOSE']]
df_loss_limit_final.to_csv(r'C:\Users\malcolm.udoh\Output_Files\Rutgers_data_pull_2023\Rutgers_2023_BetMGM_NJ_RG_Features_Loss_Limit.csv', index = False)

### Wager

In [ ]:
### Comes from Austria Team

### Snowflake

### Cool-Offs

In [ ]:
query = '''with test_account_player_id as (
select distinct dp.player_id
from edlprodviews.edlprodviewsusabi.dim_player dp
inner join edlprodviews.edlprodviewsusabi.dim_player_account dpa
    on dp.player_id = dpa.player_id
left join edlprodviews.edlprodviewsusabi.dim_cashier_category dcc
    on dpa.wh_cashier_category_cd = dcc.wh_cashier_category_cd
left join edlprodviews.edlprodviewsusabi.player_extension pe
    on dp.player_id = pe.player_id
left join edlprodviews.edlprodviewsusabi.dim_player_contact_email dpi
    on dp.player_id = dpi.player_id
where 1 = 1
    and dcc.Cashier_Category_Desc not in ('Closed','PermanentlyClosed','PermenantlyClosed')
    and (dcc.cashier_category_desc in ('TestAccount')
         or trim(lower(pe.test_account_yn)) = 'y'
         or trim(lower(cast(dpi.email_domain_txt as varchar))) like ('%bwinparty%')
         or trim(lower(cast(dpi.email_domain_txt as varchar))) like ('%ivycomptech%')
         or trim(lower(cast(dpi.email_domain_txt as varchar))) like ('%gvcgroup%')
         or trim(lower(cast(dpi.email_domain_txt as varchar))) like ('%yopmail%')
         or trim(lower(cast(dpi.email_domain_txt as varchar))) like ('%mgm%')
         or trim(lower(cast(dpi.email_domain_txt as varchar))) like ('%ladbrokescoral1%')
         or trim(lower(cast(dpi.email_domain_txt as varchar))) like ('%internalgvc%')
         or trim(lower(cast(dp.Login_Name_Txt as varchar))) like ('%beta')
         or trim(lower(cast(dp.Login_Name_Txt as varchar))) like ('%test')
         )
)

select dupi.dupi,
       ccl.effective_from_timestamp as created_dt,
       ccl.effective_to_timestamp as end_dt,
       DATEDIFF('day', ccl.effective_from_timestamp, ccl.effective_to_timestamp) period
from edlprodviews.edlprodviewsusabi.cashier_category_log ccl
inner join edlprodviews.edlprodviewsusabi.dim_player dp
	on ccl.player_id = dp.player_id
inner join edlprodviews.edlprodviewsusabi.dim_skin ds
    on dp.registration_wh_skin_id = ds.wh_skin_id
    and ds.state = ('NJ')
    and ds.front_end_cd in ('mj','bj','pj','wo')
left join edlprodviews.edlprodviewsusabi.PLAYER_DUPI_UAT dupi
    on dp.player_id = dupi.player_id
inner join edlprodviews.edlprodviewsusabi.dim_cashier_category dcc
	on ccl.wh_cashier_category_cd = dcc.wh_cashier_category_cd
	and dcc.cashier_category_desc = 'OnCool-Off'
where 1 = 1
    and dp.registration_date <= date('2023-12-31')
    and ccl.effective_from_timestamp <= timestamp '2023-12-31 23:59:59'
    and ccl.effective_to_timestamp >= timestamp '2023-01-01 00:00:00'
    and ccl.player_id not in (select *
                              from test_account_player_id)
'''
df_on_cooloff = pd.read_sql(query, conn_sf_dev)
df_on_cooloff.columns = map(str.upper, df_on_cooloff.columns)
print(df_on_cooloff.shape[0],'\n')
df_on_cooloff.to_csv(r'C:\Users\malcolm.udoh\Output_Files\Rutgers_data_pull_2023\Rutgers_BetMGM_NJ_2023_CoolOff.csv', index = False)
df_on_cooloff.head(1000).to_csv(r'C:\Users\malcolm.udoh\Output_Files\Rutgers_data_pull_2023\Rutgers_BetMGM_NJ_2023_CoolOff_sample.csv', index = False)
df_on_cooloff.head()

### Account Suspensions

In [ ]:
query = '''
/*Acct_Suspensions*/
select case
           when dupi.dupi is not null then dupi.dupi
           else cast(dp.player_id as varchar(20))
       end as DUPI,
       --UPPER(date_format(ccl.effective_from_timestamp, '%d%b%y:%T')) Suspension_DT,
       ccl.effective_from_timestamp as Suspension_DT,
       case
           when ccl.effective_to_date = '2999-12-31' then 'indefinite'
           else to_varchar(DATEDIFF('day', ccl.effective_from_timestamp, ccl.effective_to_timestamp))
       end as "Length",
      --Reason for suspension
      case
          WHEN (LOWER(to_varchar(ccl.comments))) LIKE ('%account under review by nj risk fraud%') THEN 'GeoComply SAR'
          WHEN (LOWER(to_varchar(ccl.comments))) LIKE ('%rtr scoring (system)%') THEN 'RTR Scoring Rule'
          WHEN (LOWER(to_varchar(ccl.comments))) LIKE ('%trading analyst%permanently closed%') THEN 'Trading Team Closure Request'
          WHEN (LOWER(to_varchar(ccl.comments))) LIKE ('%cb applied (system)%') THEN 'Chargeback Rule'
          WHEN (LOWER(to_varchar(ccl.comments))) LIKE ('%due to 5 consecutive failed ach deposits%') THEN 'ACH Deposit Rule'
          --WHEN (LOWER(to_varchar(ccl.comments) LIKE ('%cool-off linked to%') OR LOWER(to_varchar(ccl.comments)) LIKE ('%rg linkages%') THEN 'RG Linkage Rule'
          WHEN (LOWER(to_varchar(ccl.comments))) LIKE ('%dormancy regulation%') THEN 'Dormancy Regulation(s)'
          WHEN (LOWER(to_varchar(ccl.comments))) LIKE ('%financial fraud%') THEN 'Financial Fraud'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%unauthorized account use%') OR LOWER(to_varchar(ccl.comments)) LIKE ('%unauthorized use%')) THEN 'Unauthorized Account Use'
          WHEN (LOWER(to_varchar(ccl.comments))) LIKE ('%device%') THEN 'Device Sharing'
          WHEN (LOWER(to_varchar(ccl.comments))) LIKE ('%susp for 180 days (system)%') THEN 'Inactivity Rule' --180 days
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%not in% name%') 
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%name on card%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%3rd party%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%sharing financial%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%sharing% bank%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%sharing% paypal%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%name mismatch%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%shared% financial%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%third party%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%financial%sharing%')) THEN '3rd Party and/or Shared Financial(s)'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%sharing%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%multi account%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%linked to%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%multi-accounting%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%linked to%')) THEN 'Account Sharing/Linkages'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%both sides%') OR LOWER(to_varchar(ccl.comments)) LIKE ('%betting abuse%')) THEN 'Betting Both Sides'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%bonus abuse%') OR LOWER(to_varchar(ccl.comments)) LIKE ('%abusing bonus%')) THEN 'Bonus Abuse'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%money dumping%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%dumping money%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%by poker team%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%poker fraud%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%collusion%')) THEN 'Money Dumping/Collusion'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%identity theft%') OR LOWER(to_varchar(ccl.comments)) LIKE ('%id theft%')) THEN 'Identity Theft'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%terms%conditions%') OR LOWER(to_varchar(ccl.comments)) LIKE ('%violation%terms%')) THEN 'Terms & Conditions Violation'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%out of state%') OR LOWER(to_varchar(ccl.comments)) LIKE ('oos%') OR LOWER(to_varchar(ccl.comments)) LIKE ('%out-of-state%')) THEN 'Out-of-State'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%prohibited software%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%banned software%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%rooted device%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%fake location%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%geocomply%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%blocked%software%')) THEN 'GeoComply Block/Issue'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%addicted%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%rg team%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%responsible gaming%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%rg% keywords%')) THEN 'Responsible Gaming'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('% cb%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%cb %')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%cbs %')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%chargeback%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%outstanding% balance%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%negative% balance%')) THEN 'Outstanding Balance'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%pending% id%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%pending% ba%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%scid %')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%poa %')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%cc% copies%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%pending% doc%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%req% doc%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%req% id%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%ok to% open%')) THEN 'Pending Documents'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%request% closure%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%per player% request%')
                OR LOWER(to_varchar(ccl.comments)) LIKE ('%req% closure%')) THEN 'Account Closure Request'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%alert %') OR LOWER(to_varchar(ccl.comments)) LIKE ('% triggered%')) THEN 'Alert Triggered'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%csm %')) THEN 'Alert Triggered'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%fraud ring%') OR (dcc.cashier_category_desc = 'Fraud')) THEN 'Fraud Activity/Linkages'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%need to verify%') OR LOWER(to_varchar(ccl.comments)) LIKE ('%dfr%')) THEN 'Need to verify (Misc)'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%report%')) THEN 'Reporting'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%friendly fraud%')) THEN 'Friendly Fraud'
          WHEN (LOWER(to_varchar(ccl.comments)) LIKE ('%regulation%')) THEN 'Misc. Regulation'
          WHEN (LOWER(to_varchar(ccl.comments)) like ('%dormant%')) then 'Dormancy'
          ELSE 'Other'
       END AS Reason, 
       --UPPER(date_format(ccl.effective_to_timestamp, '%d%b%y:%T')) Suspension_removed_DT
       ccl.effective_to_timestamp as Suspension_removed_DT
from edlprodviews.edlprodviewsusabi.dim_player dp
inner join edlprodviews.edlprodviewsusabi.dim_skin ds
    on dp.registration_wh_skin_id = ds.wh_skin_id
    and ds.state = ('NJ')
   -- and dp.front_end_cd in ('mj','bj','pj', 'wo')
inner join edlprodviews.edlprodviewsusabi.cashier_category_log ccl
    on ccl.player_id = dp.player_id
    and (ccl.effective_from_timestamp between '2023-01-01' and '2023-12-31'
        or (ccl.effective_from_timestamp < '2024-01-01'
            and ccl.effective_to_timestamp >= '2023-01-01'))
inner join edlprodviews.edlprodviewsusabi.dim_cashier_category dcc
    on ccl.wh_cashier_category_cd = dcc.wh_cashier_category_cd
    and dcc.cashier_category_desc in ('Blocked', 'Closed', 'ClosedInactive', 'FEClosed', 'Fraud',
                                      'Inactive', 'PermanentlyClosed', 'Suspicious', 'Suspended')
left join edlprodviews.edlprodviewsusabi.PLAYER_DUPI_UAT dupi
    on dp.player_id = dupi.player_id
where 1 = 1
    and dp.registration_date < '2024-01-01'
    and DATEDIFF('second', ccl.effective_from_timestamp, ccl.effective_to_timestamp) > 60
order by dp.login_name_txt
'''

df_acct_suspension = pd.read_sql(query, conn_sf_dev)
df_acct_suspension.columns = map(str.upper, df_acct_suspension.columns)
print(df_acct_suspension.shape[0],'\n')
df_acct_suspension.head()

In [ ]:
df_acct_suspension.to_csv(r'C:\Users\malcolm.udoh\Output_Files\Rutgers_data_pull_2023\Rutgers_BetMGM_NJ_2023_Account_Suspension.csv', index = False)

### Money Deposited

In [4]:
query = '''
select dupi.dupi as DUPI,
       --UPPER(date_format(d.Attempt_Timestamp, '%d%b%y:%T')) Deposit_DT,
       --To_varchar(d.Attempt_Timestamp,'YYYY-MM-DD HH24:MI:SS') as Deposit_DT,
       d.Attempt_Timestamp as Deposit_DT,
       d.Txn_Amt AS Amount,
       processor_name_txt as "Method" 
       
from EDLPRODVIEWS.EDLPRODVIEWSUSABI.deposit d 
inner join EDLPRODVIEWS.EDLPRODVIEWSUSABI.Dim_Processor p 
    ON d.WH_Processor_ID = p.Wh_Processor_Id 
inner join EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_player dp 
    on dp.player_id = d.entity_id 
left join EDLPRODVIEWS.edlprodviewsusabi.PLAYER_DUPI_UAT dupi
    on dp.player_id = dupi.player_id
inner join EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_skin ds 
    on dp.registration_wh_skin_id = ds.wh_skin_id 
    and ds.state = ('NJ')
    and dp.front_end_cd in ('mj','bj','pj','wo') 
where 1 = 1
    and d.Attempt_Date between
        '2023-01-01' and '2023-12-31' 
    and d.Txn_Amt > 0.01
    and d.settled_acc_curr_amt = d.attempted_acc_curr_amt
order by 2
'''
df_dep = pd.read_sql(query, conn_sf_dev)
df_dep.columns = map(str.upper, df_dep.columns)
df_dep['AMOUNT'] = df_dep['AMOUNT'].astype(float)
print(df_dep.shape[0],'\n')
df_dep.head()

14658836 



,DUPI,DEPOSIT_DT,AMOUNT,METHOD
0,9f65d1895471e8e157040505ccc21489a75221bd,2023-01-01 00:00:03,50.0,VISA
1,4e6d5d23f3310cb7bb03648005473c7475aa11ef,2023-01-01 00:00:12,145.0,VISA
2,735a5c2a078dbd2dd08cf65be907478915080280,2023-01-01 00:00:12,260.0,VISA
3,87cc46476c2a958f1fb095c20b36f5085fb9baab,2023-01-01 00:00:13,18.0,VISA
4,ff67403297594f51266ec8614e99b87bef9f9e01,2023-01-01 00:00:15,131.0,VISA


In [6]:
df_dep.to_csv(r'C:\Users\malcolm.udoh\Output_Files\Rutgers_data_pull_2023\2023_Rutgers_BetMGM_NJ_Money_Deposited.csv', index = False)

#### Money Withdrawn

In [ ]:
query = '''
/*Withdrawals*/
select dupi.dupi as DUPI,
       --UPPER(date_format(d.Attempt_Timestamp, '%d%b%y:%T')) Deposit_DT,
       d.txn_timestamp as Withdrawal_DT,
       d.withdrawal_amt_usd AS Amount,
       trim(to_varchar(processor_name_txt)) as "Method"
       
from EDLPRODVIEWS.EDLPRODVIEWSUSABI.f_player_cashier_txn d 
inner join EDLPRODVIEWS.EDLPRODVIEWSUSABI.Dim_Processor p 
    ON d.WH_Processor_ID = p.Wh_Processor_Id 
inner join EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_player dp 
    on dp.player_id = d.player_id 
left join EDLPRODVIEWS.EDLPRODVIEWSUSABI.PLAYER_DUPI_UAT dupi
    on dp.player_id = dupi.player_id
inner join EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_skin ds 
    on dp.registration_wh_skin_id = ds.wh_skin_id 
    and ds.state = ('NJ')
    and dp.front_end_cd in ('mj','bj','pj','wo') 
where 1 = 1
    and d.txn_date between
        '2023-01-01' and '2023-12-31'
    and d.withdrawal_qty = 1
order by 2 
'''
df_wd = pd.read_sql(query, conn_sf_dev)
df_wd.columns = map(str.upper, df_wd.columns)
df_wd['AMOUNT'] = df_wd['AMOUNT'].astype(float)
print(df_wd.shape[0],'\n')
df_wd.head()

In [ ]:
df_wd.to_csv(r'C:\Users\malcolm.udoh\Output_Files\Rutgers_data_pull_2023\2023_Rutgers_BetMGM_NJ_Money_Withdrawn.csv', index = False)

### Bonus

In [ ]:
query = '''
SELECT  dupi.dupi 
        , TO_CHAR(eat.txn_timestamp, 'YYYY-MM-DD HH24:MI:SS') as transaction_dt
        , SUM(CASE 
                WHEN (att.src_txn_type_cd IN ('212', '15', '46') AND ZEROIFNULL(pb.Bonus_Restriction_Type_CD) = 2) 
                    OR att.src_txn_type_cd IN ('59'/*Poker*/, '367', '375'/*Bingo */) 
                        THEN eat.acc_curr_amt 
                ELSE 0
              END) Non_Cashable
        , SUM(CASE 
                WHEN eat.account_type_cd = 'pok' AND att.profit_loss_ind = -1 AND ((att.src_txn_type_cd IN ('212', '15', '46') 
                     AND ZEROIFNULL(pb.Bonus_Restriction_Type_CD) <> 2) 
                     OR (att.src_txn_type_cd NOT IN ('59', '81', '212', '15', '46', '246', '367', '375'))) 
                         THEN acc_curr_amt 
                ELSE 0
              END) Cashable
        , (SUM(CASE 
                WHEN att.account_type_cd = 'pok' AND att.src_txn_type_cd in ('47', '217', '216', '60') AND ZEROIFNULL(pb.Bonus_Restriction_Type_CD) = 2 THEN eat.acc_curr_amt 
                ELSE 0 
              END) 
        + SUM(CASE 
                WHEN att.account_type_cd = 'pok' AND att.src_txn_type_cd in ('47', '217', '216', '60') AND ZEROIFNULL(pb.Bonus_Restriction_Type_CD) <> 2 THEN eat.acc_curr_amt 
                ELSE 0 
              END)) AS Forfeited
FROM    EDLPRODVIEWS.edlprodviewsusabi.entity_account_transaction eat 
JOIN    EDLPRODVIEWS.edlprodviewsusabi.Dim_Player p 
    ON p.player_id = eat.entity_id
left join EDLPRODVIEWS.EDLPRODVIEWSUSABI.PLAYER_DUPI_UAT dupi
    on p.player_id = dupi.player_id
JOIN    EDLPRODVIEWS.edlprodviewsusabi.account_txn_type att 
    ON eat.wh_account_txn_type_cd = att.wh_account_txn_type_cd
LEFT JOIN    EDLPRODVIEWS.edlprodviewsusabi.player_bonus pb
    ON eat.wh_account_txn_id = pb.Bonus_WH_Account_Txn_ID
    AND pb.Bonus_Offered_Date >= '2013-10-15' AND pb.platform_cd = eat.Platform_CD --AND pb.Bonus_Issued_Date = date -2
    AND pb.Player_Id = eat.Entity_Id
LEFT JOIN    EDLPRODVIEWS.edlprodviewsusabi.bonus b 
    on b.wh_bonus_id = pb.wh_bonus_id 
LEFT JOIN    EDLPRODVIEWS.edlprodviewsusabi.bonus_rule br 
    on br.wh_bonus_rule_id=b.wh_bonus_rule_id
LEFT JOIN    EDLPRODVIEWS.edlprodviewsusabi.Bonus_Rule_Criteria BRC 
    ON BR.WH_Bonus_Rule_ID = BRC.WH_Bonus_Rule_ID 
    AND Bonus_Rule_Criteria_Type_CD = 2
WHERE   1=1
    AND     p.front_end_cd IN ('bj','mj','pj', 'wo')
    AND     eat.Txn_Date >= date '2023-01-01'
    AND     eat.txn_date <= date '2023-12-31'
    AND     att.src_txn_type_cd IN (/*Issued */'59', '212', '15', '46', '367', '375'
                                    /*Expired */ , '47', '217', '216', '60') 
GROUP BY 1,2
order by 2
'''

df_bonus = pd.read_sql(query, conn_sf_dev)
df_bonus.columns = map(str.upper, df_bonus.columns)
df_bonus['NON_CASHABLE'] = df_bonus['NON_CASHABLE'].astype(float)
df_bonus['CASHABLE'] = df_bonus['CASHABLE'].astype(float)
df_bonus['FORFEITED'] = df_bonus['FORFEITED'].astype(float)
print(df_bonus.shape[0],'\n')
df_bonus.head()

In [ ]:
df_bonus.to_csv(r'C:\Users\malcolm.udoh\Output_Files\Rutgers_data_pull_2023\2023_Rutgers_BetMGM_NJ_Bonus.csv', index = False)

### Poker Cash/Play games

In [ ]:
query = '''
--/*Poker Cash Game*/
select dupi.dupi as DUPI,
       to_char(pcg.visit_start_timestamp, 'YYYY-MM-DD HH24:MI:SS') round_dt,
       ds.State || ' ' || ds.Primary_Brand_Cd as Skin,
       pcg.bet_amt_usd as bet_amount,
       pcg.win_amt_usd as win_amount,
       pcg.rake_amt_usd as rake
from edlprodviews.EDLPRODVIEWSUSABI.f_player_poker_cash_game_visit pcg
left join edlprodviews.edlprodviewsusabi.PLAYER_DUPI_UAT dupi
    on pcg.player_id = dupi.player_id
inner join edlprodviews.EDLPRODVIEWSUSABI.dim_skin ds
    on pcg.wh_skin_id = ds.wh_skin_id
    and ds.state = ('NJ')
    and trim(lower(ds.front_end_cd)) in ('mj','bj','pj','wo')
where 1 = 1
    and pcg.visit_start_timestamp between
        '2023-01-01' and '2023-12-31'
order by 2
'''
df_pcg = pd.read_sql(query, conn_sf_dev)
df_pcg.columns = map(str.upper, df_pcg.columns)
for y in [x for x in df_pcg.columns if x.find('AMOUNT') > -1] + ['RAKE']:
    df_pcg[y] = df_pcg[y].astype(float)
print(df_pcg.shape[0],'\n')
df_pcg.head()

In [ ]:
df_pcg.to_csv(r'C:\Users\malcolm.udoh\Output_Files\Rutgers_data_pull_2023\Rutgers_BetMGM_NJ_2023_Poker_cash_games.csv', index = False)
#BetMGM_NJ_2023_Rutgers_Poker_Play_20230101_20231231.csv'

### Poker Tourney

In [ ]:
#Snowflake
query_sf = '''
select --ppt.player_id, 
       dupi.dupi as DUPI,
       --TRUNCATE(ppt.Visit_Start_Timestamp),
       To_varchar(ppt.Visit_Start_Timestamp, 'YYYY-MM-DD HH24:MI:SS') starting_dt,
       To_char(ppt.Visit_End_Timestamp, 'YYYY-MM-DD HH24:MI:SS') closing_dt,  
       pt.Guaranteed_Prize_Amt as GUARANTEED_PRIZE,
       pti.buy_in_amt as Entry_Fee,
       ppt.Take_From_Table_Amt_USD as Winnings,
       pti.Fee_amt as rake
       --ds.State || ' ' || ds.Primary_Brand_Cd as Skin
from EDLPRODVIEWS.EDLPRODVIEWSUSABI.f_player_poker_trny_visit ppt
left join EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_poker_trny_table_info pti
    ON pti.wh_poker_trny_table_info_id = ppt.wh_poker_trny_table_info_id
left join EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_poker_tournament pt
    ON pt.wh_tournament_id = ppt.wh_tournament_id
left join EDLPRODVIEWS.EDLPRODVIEWSUSABI.PLAYER_DUPI_UAT dupi
    on ppt.player_id = dupi.player_id
inner join EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_skin ds
    on ppt.wh_skin_id = ds.wh_skin_id
    and ds.state = ('NJ')
    and trim(lower(ds.front_end_cd)) in ('mj','bj','pj','wo')
where 1 = 1
    and ppt.rank_date between
        DATE('2023-01-01') and DATE('2023-12-31')
order by 2
'''

df_ppt = pd.read_sql(query_sf, conn_sf_dev)
df_ppt.columns = map(str.upper, df_ppt.columns)
#for y in [#'GUARANTEED_PRIZE',
        #  'ENTRY_FEE','RAKE','WINNINGS']:
   # df_ppt[y] = df_ppt[y].astype(float)
print(df_ppt.shape[0],'\n')
df_ppt.head()

In [ ]:
df_ppt.to_csv(r'C:\Users\malcolm.udoh\Output_Files\Rutgers_data_pull_2023\Rutgers_BetMGM_NJ_2023_Poker_Tournament.csv', index = False)

### Self Exclusion

In [ ]:
query = '''
select dupi.dupi, 
       upper(TO_CHAR(ccl.effective_from_timestamp, 'DDMonYY:HH24:MI:SS')) as self_exclusion_dt,
       case
        when effective_to_date >= current_date() then 'ongoing'
        else cast(datediff(day, effective_from_date, effective_to_date) as varchar)
        end as period,
       upper(TO_CHAR(ccl.effective_to_timestamp, 'DDMonYY:HH24:MI:SS')) as removal_dt
from EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_player dp
left join EDLPRODVIEWS.EDLPRODVIEWSUSABI.PLAYER_DUPI_UAT dupi
    on dp.player_id = dupi.player_id
inner join EDLPRODVIEWS.EDLPRODVIEWSUSABI.cashier_category_log ccl
    on dp.player_id = ccl.player_id
    and ccl.effective_from_date <= '2023-12-31'
    and ccl.effective_to_date >= '2023-01-01'
inner join EDLPRODVIEWS.EDLPRODVIEWSUSABI.dim_cashier_category dcc
    on ccl.wh_cashier_category_cd = dcc.wh_cashier_category_cd
    and dcc.cashier_category_desc = 'RGClosed'
where 1 = 1
    and dp.front_end_cd in ('bj','pj','mj', 'wo')
    and dp.registration_date <= '2023-12-31'
order by 1
'''

df_se = pd.read_sql(query, conn_sf_dev)
df_se.columns = map(str.upper, df_se.columns)
print(df_se.shape[0],'\n')
df_se.to_csv(r'C:\Users\malcolm.udoh\Output_Files\Rutgers_data_pull_2023\2023_Rutgers_BetMGM_NJ_Self_Exclusion.csv', index = False)
df_se.head()

In [ ]:
for x in [conn_sf, conn_sf_dev]:
    try:
        x.close()
    except:
        pass